## TC 5033
### Text Generation

<br>

#### Activity 4: Building a Simple LSTM Text Generator using WikiText-2
<br>

- Objective:
    - Gain a fundamental understanding of Long Short-Term Memory (LSTM) networks.
    - Develop hands-on experience with sequence data processing and text generation in PyTorch. Given the simplicity of the model, amount of data, and computer resources, the text you generate will not replace ChatGPT, and results must likely will not make a lot of sense. Its only purpose is academic and to understand the text generation using RNNs.
    - Enhance code comprehension and documentation skills by commenting on provided starter code.
    
<br>

- Instructions:
    - Code Understanding: Begin by thoroughly reading and understanding the code. Comment each section/block of the provided code to demonstrate your understanding. For this, you are encouraged to add cells with experiments to improve your understanding

    - Model Overview: The starter code includes an LSTM model setup for sequence data processing. Familiarize yourself with the model architecture and its components. Once you are familiar with the provided model, feel free to change the model to experiment.

    - Training Function: Implement a function to train the LSTM model on the WikiText-2 dataset. This function should feed the training data into the model and perform backpropagation. 

    - Text Generation Function: Create a function that accepts starting text (seed text) and a specified total number of words to generate. The function should use the trained model to generate a continuation of the input text.

    - Code Commenting: Ensure that all the provided starter code is well-commented. Explain the purpose and functionality of each section, indicating your understanding.

    - Submission: Submit your Jupyter Notebook with all sections completed and commented. Include a markdown cell with the full names of all contributing team members at the beginning of the notebook.
    
<br>

- Evaluation Criteria:
    - Code Commenting (60%): The clarity, accuracy, and thoroughness of comments explaining the provided code. You are suggested to use markdown cells for your explanations.

    - Training Function Implementation (20%): The correct implementation of the training function, which should effectively train the model.

    - Text Generation Functionality (10%): A working function is provided in comments. You are free to use it as long as you make sure to uderstand it, you may as well improve it as you see fit. The minimum expected is to provide comments for the given function. 

    - Conclusions (10%): Provide some final remarks specifying the differences you notice between this model and the one used  for classification tasks. Also comment on changes you made to the model, hyperparameters, and any other information you consider relevant. Also, please provide 3 examples of generated texts.



#### Import libraries

In [1]:
import numpy as np
#PyTorch libraries
import torch
import torchtext
from torchtext.datasets import WikiText2
# Dataloader library
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data.dataset import random_split
# Libraries to prepare the data
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.functional import to_map_style_dataset
# neural layers
from torch import nn
from torch.nn import functional as F
import torch.optim as optim
from tqdm import tqdm

import random

In [2]:
# Use GPU if available
#device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = 'mps' if torch.backends.mps.is_available() else 'cpu'     # for mac with apple silicon
print(device)

mps


### Load the data

In [3]:
train_dataset, val_dataset, test_dataset = WikiText2()

#### Get the tokenizer

The tokenizer function will be used to convert the text into a list of tokens.

In [4]:
tokeniser = get_tokenizer('basic_english')
def yield_tokens(data):
    for text in data:
        yield tokeniser(text)

The following block of code will create a vocabulary of tokens from the train dataset.

The parameter `specials` is used to specify special tokens that will be added to the vocabulary. In this case, we are adding four special tokens:

 * `<unk>`: for unknown words, not found in the training dataset.
 * `<pad>`: for padding sequences to the same length.
 * `<bos>`: for marking the begining of a sequence.
 * `<eos>`: for marking the end of a sequence.

In [5]:
# Build the vocabulary
vocab = build_vocab_from_iterator(yield_tokens(train_dataset), specials=["<unk>", "<pad>", "<bos>", "<eos>"])
#set unknown token at position 0
vocab.set_default_index(vocab["<unk>"])

#### Process the data

The function `data_process` returns the input tensor and the target tensor that will be used to train the LSTM model.

Arguments: 
* `raw_text_iter` iterator over the text dataset.
* `seq_length` length of the sequence for the LSTM model, default = 50.

First, the function will tokenize each item using `tokenizer` then convert the tokens to their corresponding indices in the vocabulary using `vocab`, and then convert the resulting list of indices to a tensor.

Then, the function will remove empty tensors and concatenate the remaining tensors into a single tensor using `torch.cat`.

Finally, the function will return two tensors: the first tensor is created by trimming the `data` tensor to a legth that is a multiple of the sequence length and reshaping it into a 2D tensor. The second tennsor is created by shifting the `data` tensor by one position to the right, trimming its length to the same length as the first tensor and reshaping into a 2D tensor.


In [6]:
seq_length = 50
def data_process(raw_text_iter, seq_length = 50):
    data = [torch.tensor(vocab(tokeniser(item)), dtype=torch.long) for item in raw_text_iter]
    data = torch.cat(tuple(filter(lambda t: t.numel() > 0, data))) #remove empty tensors
#     target_data = torch.cat(d)
    return (data[:-(data.size(0)%seq_length)].view(-1, seq_length), 
            data[1:-(data.size(0)%seq_length-1)].view(-1, seq_length))  

# # Create tensors for the training set
x_train, y_train = data_process(train_dataset, seq_length)
x_val, y_val = data_process(val_dataset, seq_length)
x_test, y_test = data_process(test_dataset, seq_length)

The `TensorDataset` class is used to create a dataset of tensors. The dataset will be used to create a data loader that will be used to feed the data to the LSTM model during training.

In [7]:
train_dataset = TensorDataset(x_train, y_train)      
val_dataset = TensorDataset(x_val, y_val)
test_dataset = TensorDataset(x_test, y_test)

In [8]:
print(len(train_dataset), len(val_dataset), len(test_dataset))

40999 4288 4837


#### Create data loaders

In [9]:
batch_size = 64  # choose a batch size that fits your computation resources
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, drop_last=True)

### Build the LSTM model

In [10]:
# Define the LSTM model
# Feel free to experiment
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers):
        super(LSTMModel, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embed_size)
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, text, hidden):
        embeddings = self.embeddings(text)
        output, hidden = self.lstm(embeddings, hidden)
        decoded = self.fc(output)
        return decoded, hidden

    def init_hidden(self, batch_size):

        return (torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device),
                torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device))



vocab_size = len(vocab) # vocabulary size
emb_size = 256 # embedding size
neurons = 300 # the dimension of the feedforward network model, i.e. # of neurons 
num_layers = 2 # the number of nn.LSTM layers
model = LSTMModel(vocab_size, emb_size, neurons, num_layers)


In [11]:
def train(model, epochs, optimiser):
    '''
    The following are possible instructions you may want to conside for this function.
    This is only a guide and you may change add or remove whatever you consider appropriate
    as long as you train your model correctly.
        - loop through specified epochs
        - loop through dataloader
        - don't forget to zero grad!
        - place data (both input and target) in device
        - init hidden states e.g. hidden = model.init_hidden(batch_size)
        - run the model
        - compute the cost or loss
        - backpropagation
        - Update paratemers
        - Include print all the information you consider helpful
    
    ''' 
    
    model = model.to(device=device)                         # move the model parameters to GPU
    model.train()                                           # put the model in training mode
    
    for epoch in range(epochs):
        for i, (data, targets) in enumerate((train_loader)):            # loop through the data loader
            data = data.to(device=device)                               # place input data in device
            targets = targets.to(device=device)                         # place targets in device
            
            hidden = model.init_hidden(batch_size)                      # init hidden states
            outputs, hidden = model(data, hidden)                       # forward pass
            loss = F.cross_entropy(outputs.view(-1, vocab_size), targets.view(-1))  # compute loss  

            optimiser.zero_grad()                            # zero grad
            loss.backward()                                  # backpropagation
            optimiser.step()                                 # update parameters


        print(f'Epoch: {epoch+1}, Loss: {loss.item()}')
                


In [12]:
# Call the train function
#loss_function = nn.CrossEntropyLoss()
lr = 0.0005
epochs = 10
optimiser = optim.Adam(model.parameters(), lr=lr)
train(model, epochs, optimiser)

Epoch: 1, Loss: 6.7527971267700195
Epoch: 2, Loss: 6.402383327484131
Epoch: 3, Loss: 6.11318826675415
Epoch: 4, Loss: 5.939584732055664
Epoch: 5, Loss: 5.92225456237793
Epoch: 6, Loss: 6.011138916015625
Epoch: 7, Loss: 5.824009895324707
Epoch: 8, Loss: 5.675213813781738
Epoch: 9, Loss: 5.588720798492432
Epoch: 10, Loss: 5.495991230010986


In [32]:
def generate_text(model, start_text, num_words, temperature=1.0):
    
    model.eval()
    words = tokeniser(start_text)
    hidden = model.init_hidden(1)
    for i in range(0, num_words):
        x = torch.tensor([[vocab[word] for word in words[i:]]], dtype=torch.long, device=device)
        y_pred, hidden = model(x, hidden)
        last_word_logits = y_pred[0][-1]
        p = (F.softmax(last_word_logits / temperature, dim=0).detach()).to(device='cpu').numpy()
        word_index = np.random.choice(len(last_word_logits), p=p)
        words.append(vocab.lookup_token(word_index))

    return ' '.join(words)
    
    

# Generate some text
print(generate_text(model, start_text="I like", num_words=100))


i like he gifts and real 700 kilometers was investigating any of this portion of the cause of the frequency . fey is selection on the natural domestic pacific three nambu bat lite . another antagonist reached rating , they were described as great club with the character of 24 fatalities . 2s by <unk> <unk> the newcastle @-@ texts edition in the fourth hydrogen lake star ( san in overall ) hall of adam = = creator years geography allah and <unk> boats under the series video bombardments . other corresponds which joey funding by the mortal visiting organisms onward ,


In [33]:
start_text = "Hello my name is"

print(generate_text(model, start_text=start_text, num_words=50))

hello my name is not presented , which had associated a class . he found shipyard a rubbers among venus . sir <unk> of eva began frank charlie and alkan recently , many his have new clear , and his remaster and mummies to some taylor , and tom geoff on the season .


In [37]:
start_text = "In the city of"

print(generate_text(model, start_text=start_text, num_words=30))

in the city of 1975 , karan in the late 2003 @-@ income layer in a one territory . = = tropical gracie is splash high in consolidating , the <unk> shattering with an


In [35]:
start_text = "The president said"

print(generate_text(model, start_text=start_text, num_words=20))

the president said . offices of the inaugural news and the iii working on the commonwealth <unk> , the font to accommodate half
